In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.service import Service
import time
import pandas as pd
import requests

In [2]:
url = 'https://www.zomato.com/belgaum/restaurants'
s = Service("Add you chrome driver location")

In [3]:
driver = webdriver.Chrome(service = s)
driver.get(url)
time.sleep(6)

In [4]:
session = requests.Session()
session.headers.update({
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept-Language': 'en-US,en;q=0.9',
    'Accept-Encoding': 'gzip, deflate, br',
    'Referer': 'https://www.zomato.com/'
})

response = session.get(url)
print(response.status_code)

200


In [5]:
height = driver.execute_script("return document.body.scrollHeight")

In [6]:
while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
    time.sleep(6)
    new_height = driver.execute_script("return document.body.scrollHeight")
    if height == new_height:
        break
    height = new_height

In [7]:
html = driver.page_source

In [8]:
driver.quit()

In [9]:
soup = BeautifulSoup(html, 'html.parser')

In [10]:
containers = soup.find_all('div', class_='jumbo-tracker')
len(containers)

505

In [11]:
cat_tags = soup.find_all('p', class_=lambda x: x and 'sc-faQXZc' in x)
categories_with_counts = [p.get_text(strip=True) for p in cat_tags]

In [12]:
base_url = 'https://www.zomato.com'

In [13]:
# Lists to hold restaurant names and URLs
res_names = []
res_urls = []

for container in containers:
    # Extract the restaurant name
    res_name_tag = container.find('h4')  # Adjust the tag if necessary
    if res_name_tag is not None:
        res_name = res_name_tag.text.strip()
    else:
        res_name = ''  # Default to empty string if name is not found

    # Extract the restaurant URL
    url_tag = container.find('a')  # Assuming the URL is within an a tag
    if url_tag and 'href' in url_tag.attrs:
        relative_url = url_tag['href']
        absolute_url = base_url + relative_url if relative_url else ''
    else:
        absolute_url = ''  # Default to empty string if URL is not found

    # Only append if the restaurant name is not empty
    if res_name:
        res_names.append(res_name)
        res_urls.append(absolute_url)
    else:
        print(f"Skipped entry with empty name. URL: {absolute_url}")  # Log the skipped entry

# Ensure both lists have the same length
if len(res_names) != len(res_urls):
    print("Warning: Mismatch in lengths of res_names and res_urls")
    print(f"Length of res_names: {len(res_names)}, Length of res_urls: {len(res_urls)}")

Skipped entry with empty name. URL: https://www.zomato.com/belgaum/delivery/dish-pizza
Skipped entry with empty name. URL: https://www.zomato.com/belgaum/delivery/dish-biryani
Skipped entry with empty name. URL: https://www.zomato.com/belgaum/delivery/dish-burger
Skipped entry with empty name. URL: https://www.zomato.com/belgaum/delivery/dish-cake
Skipped entry with empty name. URL: https://www.zomato.com/belgaum/delivery/dish-chicken
Skipped entry with empty name. URL: https://www.zomato.com/belgaum/delivery/dish-chaat
Skipped entry with empty name. URL: https://www.zomato.com/belgaum/delivery/dish-thali
Skipped entry with empty name. URL: https://www.zomato.com/belgaum/delivery?place_name=Belgaum&dishv2_id=35798&context=delivery
Skipped entry with empty name. URL: https://www.zomato.com/belgaum/delivery/dish-rolls
Skipped entry with empty name. URL: https://www.zomato.com/belgaum/delivery/dish-momos
Skipped entry with empty name. URL: https://www.zomato.com/belgaum/delivery/dish-nood

In [14]:
df_restaurants = pd.DataFrame({
    'name': res_names,
    'url': res_urls
})

<h1>Address and Phone Number Scraper</h1>

In [16]:
import requests
from bs4 import BeautifulSoup

session = requests.Session()
res_data = []  # List of dicts with URL, Address, Phone Numbers

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36',
    'Accept-Language': 'en-US,en;q=0.9',
    'Referer': 'https://www.google.com'
}

for idx, url in enumerate(df_restaurants['url'], start=1):
    print(f"\n--- Processing URL {idx}/{len(df_restaurants['url'])} ---")
    print(f"🔗 {url}")

    try:
        response = session.get(url, headers=headers, timeout=10)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')

            # Extract Address
            address_div = soup.find('div', class_='sc-clNaTc ckqoPM')
            address = address_div.text.strip() if address_div else 'Address not available'

            # Extract Phone Numbers
            phone_numbers = []
            for a_tag in soup.find_all('a', href=True):
                if a_tag['href'].startswith('tel:'):
                    phone_numbers.append(a_tag['href'].replace('tel:', '').strip())
            if not phone_numbers:
                phone_numbers = ['No phone number found']

            res_data.append({
                'Restaurant URL': url,
                'Address': address,
                'Phone Numbers': ', '.join(phone_numbers)
            })

            print(f"📍 Address: {address}")
            print(f"📞 Phone(s): {', '.join(phone_numbers)}")

        else:
            res_data.append({
                'Restaurant URL': url,
                'Address': 'Failed to retrieve page',
                'Phone Numbers': ''
            })
            print(f"❌ Failed to fetch {url} with status code {response.status_code}")

    except requests.exceptions.RequestException as e:
        res_data.append({
            'Restaurant URL': url,
            'Address': 'Error fetching address',
            'Phone Numbers': ''
        })
        print(f"🔌 Error fetching {url}: {e}")

# Convert to DataFrame
df_restaurant_info = pd.DataFrame(res_data)
print(df_restaurant_info.head())


--- Processing URL 1/483 ---
🔗 https://www.zomato.com/belgaum/hotel-panchamruth-belgaum-locality/order
📍 Address: Maratha colony, near Arjun Talkies Tilakwadi, Belgaum
📞 Phone(s): +919845770761

--- Processing URL 2/483 ---
🔗 https://www.zomato.com/belgaum/dominos-pizza-4-belgaum-locality/order
📍 Address: CTS 3367/A/2/A & 3367/A2/B2, Ground Floor And 1st Floor, Gondhali Gali, Belgaum Locality, Belgaum
📞 Phone(s): +918312002094

--- Processing URL 3/483 ---
🔗 https://www.zomato.com/belgaum/sharma-sweets-restaurant-belgaum-locality/order
📍 Address: 1762, Kirloskar Road, Belgaum Locality, Belgaum
📞 Phone(s): +919353338962

--- Processing URL 4/483 ---
🔗 https://www.zomato.com/belgaum/hotel-virat-belgaum-locality/order
📍 Address: 4th Gate, Angol, Udyambag Road, Near KLE College, Belgaum Locality, Belgaum
📞 Phone(s): +918050148125

--- Processing URL 5/483 ---
🔗 https://www.zomato.com/belgaum/kareems-kitchen-belgaum-locality/order
📍 Address: CTS 429/B1, Congress Road, Tilakwadi, Belagavi, 

<h1> Menu Scraper</h1>

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import pandas as pd
import time
import re

# Replace with your actual ChromeDriver path
s = Service("C:/chromedriver-win64/chromedriver.exe")

# Initialize Selenium driver
driver = webdriver.Chrome(service=s)

menu_data = []

for i in range(len(res_urls)):
    restaurant_name = res_names[i]
    url = res_urls[i]

    try:
        print(f"\n--- Scraping: {restaurant_name} | {url} ---")
        driver.get(url)
        time.sleep(6)  # Wait for JavaScript to load

        # Scroll to bottom to load all content
        last_height = driver.execute_script("return document.body.scrollHeight")
        while True:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(3)
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # Find all categories and items
        category_section = soup.find('section', class_='sc-kIXKos jhHcrc')
        if not category_section:
            print("❌ Category section not found")
            continue

        categories = category_section.find_all('p', class_='sc-1hez2tp-0')
        menu_tags = soup.find_all('h4', class_='sc-cGCqpu chKhYc')
        price_tags = soup.find_all('span', class_='sc-17hyc2s-1 cCiQWA')
        type_tags = soup.find_all('svg', class_='sc-eOnLuU dlDRKy')

        print(f"✔ Categories found: {len(categories)}")
        print(f"✔ Menu items detected: {len(menu_tags)}")
        print(f"✔ Prices found: {len(price_tags)}")
        print(f"✔ Types found: {len(type_tags)}")

        menu_idx = 0

        for cat in categories:
            cat_text = cat.text.strip()
            match = re.match(r'(.*?)\s*\((\d+)\)', cat_text)
            if not match:
                continue

            category_name = match.group(1).strip()
            expected_items = int(match.group(2))
            print(f"  ➤ Category: {category_name} | Expected: {expected_items}")

            for _ in range(expected_items):
                if menu_idx >= len(menu_tags):
                    break

                item_name = menu_tags[menu_idx].text.strip()
                price = price_tags[menu_idx].text.strip() if menu_idx < len(price_tags) else 'N/A'

                # Get food type
                try:
                    icon = type_tags[menu_idx].find('use')['href']
                    if '#non-veg-icon' in icon:
                        food_type = 'Non-Veg'
                    elif '#veg-icon' in icon:
                        food_type = 'Veg'
                    else:
                        food_type = 'Other'
                except:
                    food_type = 'Other'

                menu_data.append({
                    'Restaurant Name': restaurant_name,
                    'Category': category_name,
                    'Item': item_name,
                    'Price': price,
                    'Type': food_type
                })

                menu_idx += 1

    except Exception as e:
        print(f"❌ Error while scraping {restaurant_name}: {e}")
        continue

# Close driver
driver.quit()

# Convert to DataFrame
df_menu = pd.DataFrame(menu_data)
print("\n✅ Final Menu DataFrame created:")
print(df_menu.head())



--- Scraping: Hotel Panchamruth | https://www.zomato.com/belgaum/hotel-panchamruth-belgaum-locality/order ---
✔ Categories found: 14
✔ Menu items detected: 285
✔ Prices found: 285
✔ Types found: 285
  ➤ Category: Starters | Expected: 41
  ➤ Category: Main Course | Expected: 50
  ➤ Category: Breads | Expected: 23
  ➤ Category: Rice and Biryani | Expected: 13
  ➤ Category: Fried Rice and Noodles | Expected: 27
  ➤ Category: Chinese | Expected: 5
  ➤ Category: South Indian | Expected: 27
  ➤ Category: Pizza | Expected: 3
  ➤ Category: Snacks | Expected: 5
  ➤ Category: Sandwiches | Expected: 19
  ➤ Category: Accompaniments | Expected: 9
  ➤ Category: Pav Bhaji | Expected: 8
  ➤ Category: Desserts and Beverages | Expected: 36
  ➤ Category: Soups and Salads | Expected: 19

--- Scraping: Domino's Pizza | https://www.zomato.com/belgaum/dominos-pizza-4-belgaum-locality/order ---
✔ Categories found: 15
✔ Menu items detected: 178
✔ Prices found: 178
✔ Types found: 178
  ➤ Category: Big Big Pizz

In [ ]:
df_menu.head()

In [ ]:
# First, merge the address and phone info into the main restaurant dataframe
df_full_info = pd.merge(
    df_restaurants,
    df_restaurant_info,
    left_on='url',
    right_on='Restaurant URL',
    how='left'
).drop(columns=['Restaurant URL'])  # Remove duplicate column

In [ ]:
# Now merge menu with the restaurant info using the restaurant name
df_final = pd.merge(
    df_menu,
    df_full_info,
    left_on='Restaurant Name',
    right_on='name',
    how='left'
).drop(columns=['name'])  # Drop redundant column

In [ ]:
print("\n🎉 Final Merged Data Preview:")
df_final.head()

In [ ]:
df_final.columns